In [1]:
import os
import wget
import nemo.collections.asr as nemo_asr
import numpy as np
import pandas as pd
import soundfile as sf

from nemo.collections.asr.parts.utils.speaker_utils import rttm_to_labels, labels_to_pyannote_object
from utils_for_diar import get_meta_inform, get_model
from pydub import AudioSegment
from IPython.display import Audio 
from IPython.core.display import display


data_dir = get_meta_inform()
sd_model = get_model(data_dir);

NOTE! Installing ujson may make loading annotations faster.


################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2022-04-04 08:09:38 model_utils:413] Skipped conversion for config/subconfig:
    {'manifest_filepath': '???', 'out_dir': 'outputs', 'oracle_vad': False, 'collar': 0.25, 'ignore_overlap': True, 'vad': {'model_path': 'vad_marblenet', 'external_vad_manifest': None, 'parameters': {'window_length_in_sec': 0.15, 'shift_length_in_sec': 0.01, 'smoothing': 'median', 'overlap': 0.875, 'onset': 0.8, 'offset': 0.6, 'pad_onset': 0.05, 'pad_offset': -0.1, 'min_duration_on': 0.1, 'min_duration_off': 0.4, 'filter_speech_first': True}, 'window_length_in_sec': 0.15, 'shift_length_in_sec': 0.01}, 'sp

[NeMo I 2022-04-04 08:09:38 clustering_diarizer:130] Loading pretrained vad_marblenet model from NGC
[NeMo I 2022-04-04 08:09:38 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.8.0rc0/vad_marblenet/10477085f32c378938ef41e65dc2e1b3/vad_marblenet.nemo.
[NeMo I 2022-04-04 08:09:38 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.8.0rc0/vad_marblenet/10477085f32c378938ef41e65dc2e1b3/vad_marblenet.nemo
[NeMo I 2022-04-04 08:09:38 common:704] Instantiating model from pre-trained checkpoint


[NeMo W 2022-04-04 08:09:38 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    vad_stream: false
    manifest_filepath: /home/fjia/code/manifest64/train.json
    sample_rate: 16000
    labels:
    - background
    - speech
    batch_size: 128
    num_workers: 20
    shuffle: true
    augmentor:
      shift:
        prob: 0.8
        min_shift_ms: -5.0
        max_shift_ms: 5.0
      white_noise:
        prob: 0.8
        min_level: -90
        max_level: -46
    
[NeMo W 2022-04-04 08:09:38 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    vad_stream: false
    manifest_filepath: /home/fjia/code/manifest64/validation.json
    sample_r

[NeMo I 2022-04-04 08:09:39 save_restore_connector:157] Model EncDecClassificationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.8.0rc0/vad_marblenet/10477085f32c378938ef41e65dc2e1b3/vad_marblenet.nemo.
[NeMo I 2022-04-04 08:09:39 clustering_diarizer:154] Loading pretrained titanet_large model from NGC
[NeMo I 2022-04-04 08:09:39 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.8.0rc0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2022-04-04 08:09:39 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.8.0rc0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2022-04-04 08:09:39 common:704] Instantiating model from pre-trained checkpoint


[NeMo W 2022-04-04 08:09:40 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2022-04-04 08:09:40 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2022-04-04 08:09:40 features:255] PADDING: 16
[NeMo I 2022-04-04 08:09:40 features:272] STFT using torch
[NeMo I 2022-04-04 08:09:40 label_models:99] loss is Angular Softmax
[NeMo I 2022-04-04 08:09:40 save_restore_connector:157] Model EncDecSpeakerLabelModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.8.0rc0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


In [4]:
files_for_diar = []
for root, dirs, files in os.walk('YD_wav/'):
    for file in files:
        if file.endswith('wav'):
            files_for_diar.append(root + file)
files_for_diar

["YD_wav/Ejdel'man.wav",
 'YD_wav/Orlov.wav',
 'YD_wav/Gordeeva.wav',
 'YD_wav/100.wav',
 'YD_wav/Gordon.wav',
 'YD_wav/Usachev.wav',
 'YD_wav/Tima.wav',
 'YD_wav/Jurij_xyqti.wav',
 'YD_wav/Komissarenko.wav',
 'YD_wav/Jurij_arqch.wav',
 'YD_wav/Akunin.wav']

In [5]:
sd_model.diarize(files_for_diar)

[NeMo I 2022-04-04 08:10:09 speaker_utils:58] Number of files to diarize: 11
[NeMo I 2022-04-04 08:10:09 clustering_diarizer:294] Split long audio file to avoid CUDA memory issue


[NeMo W 2022-04-04 08:11:25 nemo_logging:349] /workspace/byebye/pet_pr/nemo/collections/asr/parts/utils/vad_utils.py:118: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
      duration = librosa.get_duration(x, sr=sr)
    
[NeMo W 2022-04-04 08:11:36 nemo_logging:349] /workspace/byebye/pet_pr/nemo/collections/asr/parts/utils/vad_utils.py:118: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
      duration = librosa.get_duration(x, sr=sr)
    
[NeMo W 2022-04-04 08:11:46 nemo_logging:349] /workspace/byebye/pet_pr/nemo/collections/asr/parts/utils/vad_utils.py:118: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
      duration = librosa.get_duration(x, sr=sr)
    
[NeMo W 2022-04-04 08:11:55 nemo_logging:349] 

[NeMo I 2022-04-04 08:14:09 vad_utils:83] The prepared manifest file exists. Overwriting!
[NeMo I 2022-04-04 08:14:09 classification_models:244] Perform streaming frame-level VAD
[NeMo I 2022-04-04 08:14:09 collections:277] Filtered duration for loading collection is 0.000000.
[NeMo I 2022-04-04 08:14:09 collections:281] # 1207 files loaded accounting to # 1 labels


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1207/1207 [03:51<00:00,  5.22it/s]

[NeMo I 2022-04-04 08:18:00 clustering_diarizer:243] Generating predictions with overlapping input segments


[NeMo I 2022-04-04 08:19:20 clustering_diarizer:254] Converting frame level prediction to speech/no-speech segment in start and end times format.
[NeMo I 2022-04-04 08:19:27 clustering_diarizer:272] Subsegmentation for embedding extraction: scale0, outputs/speaker_outputs/subsegments_scale0.json
[NeMo I 2022-04-04 08:19:30 clustering_diarizer:326] Extracting embeddings for Diarization
[NeMo I 2022-04-04 08:19:31 collections:277] Filtered duration for loading collection is 0.000000.
[NeMo I 2022-04-04 08:19:31 collections:281] # 169014 files loaded accounting to # 1 labels


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [05:14<00:00, 28.62s/it]
[NeMo W 2022-04-04 08:30:37 speaker_utils:433] check if each ground truth RTTMs were present in provided manifest file. Skipping calculation of Diariazation Error Rate


[NeMo I 2022-04-04 08:30:37 clustering_diarizer:442] Outputs are saved in /workspace/byebye/pet_pr/outputs directory


In [6]:
###Save all predict in dict
pred_labels = {}
for i in files_for_diar:
    rttm_path = i.split('/')[-1][:-4:]
    pred_labels[str(rttm_path)] = {}
    pred_labels[str(rttm_path)]['raw'] = rttm_to_labels(f'outputs/pred_rttms/{rttm_path}.rttm')

In [7]:
pred_labels.keys()

dict_keys(["Ejdel'man", 'Orlov', 'Gordeeva', '100', 'Gordon', 'Usachev', 'Tima', 'Jurij_xyqti', 'Komissarenko', 'Jurij_arqch', 'Akunin'])

In [8]:
### a few modifications
for key in pred_labels.keys():
    new_data = []
    for index in range(len(pred_labels[key]['raw'])):
        if key == '100':
            if pred_labels[key]['raw'][index].split(' ')[-1] != 'speaker_1':
                new_data.append(pred_labels[key]['raw'][index])
        else:
            if pred_labels[key]['raw'][index].split(' ')[-1] != 'speaker_0':
                new_data.append(pred_labels[key]['raw'][index])
            else:
                new_data.append('drop')          
    
    pred_labels[key]['clean'] = new_data

In [9]:
### Create folder for each audio_data
for key in pred_labels.keys():
    if not os.path.exists(f'YD_short/{key}'):
        os.makedirs(f'YD_short/{key}')

In [10]:
### Cutting long audios to short audios
for key in pred_labels.keys():
    au = AudioSegment.from_wav(f'YD_wav/{key}.wav')
    for index in range(len ( pred_labels[key]['clean'] ) ):
        if pred_labels[key]['clean'][index] != 'drop':
            start, end = float(pred_labels[key]['clean'][index].split(' ')[0]), float(pred_labels[key]['clean'][index].split(' ')[1])
            au[1000 * start : 1000 * end].export(f'YD_short/{key}/short_{index}.wav', format='wav')

In [11]:
### Create Metadata
dud_wavs = []
for root, dirs, files in os.walk('YD_short/'):
    for file in files:
        if file.endswith('wav'):
            dud_wavs.append(root + '/' + file)
data = pd.DataFrame(data = dud_wavs)

In [12]:
def dur(x):
    data, samplerate = sf.read(x)
    return data.size / samplerate

In [13]:
data[1] = data[0].apply(dur)

In [14]:
### duration filtering
data = data[data[1] > 1]
data = data[data[1] < 10]
data = data.reset_index(drop=True)

In [15]:
data[1].sum()/3600

2.072066402116402

In [16]:
data[1].describe()

count    2660.000000
mean        2.804300
std         1.796141
min         1.004989
25%         1.500000
50%         2.180000
75%         3.560000
max         9.980000
Name: 1, dtype: float64

In [17]:
data.to_pickle('YuryDud_TTS_date.pkl')

In [18]:
### Проверим правильность разрезки
data[2] = data[0].apply(lambda x: x.split('/')[1])

In [19]:
data[2].unique()

array(['Akunin', "Ejdel'man", 'Orlov', 'Usachev', 'Gordon', '100',
       'Gordeeva', 'Tima', 'Komissarenko'], dtype=object)

In [20]:
for i in data[2].unique():
    index = [ np.random.randint(0, len(data[data[2] == i])) for j in range(3) ]
    print(f'Part name is {i}')
    for count in index:
        display(Audio(data[data[2] == i].reset_index(drop=True)[0][count]))

Part name is Akunin


Part name is Ejdel'man


Part name is Orlov


Part name is Usachev


Part name is Gordon


Part name is 100


Part name is Gordeeva


Part name is Tima


Part name is Komissarenko
